In [1]:
%run 00_common.ipynb

Python 3.6.1 :: Anaconda custom (64-bit)
azure-cli                         2.0.75

command-modules-nspkg              2.0.3
core                              2.0.75
nspkg                              3.0.4
telemetry                          1.0.4

Extensions:
azure-cli-iot-ext                  0.8.4

Python location '/opt/az/bin/python3'
Extensions directory '/home/hungap/.azure/cliextensions'

Python (Linux) 3.6.5 (default, Oct 11 2019, 09:04:16) 
[GCC 5.4.0 20160609]

Legal docs and information: aka.ms/AzureCliLegal


Your CLI is up-to-date.
SDK version: 1.0.39
Note, we have launched a browser for you to login. For old experience with device code, use "az login --use-device-code"
Unable to revert mtime: /usr/local/share/fonts
[10876:10896:1029/153849.433126:ERROR:browser_process_sub_thread.cc(217)] Waited 3 ms for network service
Opening in existing browser session.
You have logged in. Now let us find all the subscriptions to which you have access...
[
  {
    "cloudName": "AzureClou

In [2]:
post_login()
ws = load_workspace()

Setting subscription...
Configuring workspace...


If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Azure ML Workspace: iotedge


### Step 2: Train Model

In [27]:
# Step 1. Train 
# Load the data set
import pandas
import numpy
import pickle


from sklearn import tree
from sklearn.model_selection import train_test_split
temp_data = pandas.read_csv('temperature_data.csv')
# Load features and labels
X, Y = temp_data[['machine_temperature', 'machine_pressure', 'ambient_temperature', 'ambient_humidity']].values, temp_data['anomaly'].values
# Split data 65%-35% into training set and test set
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, random_state=0)

# Change regularization rate and you will likely get a different accuracy.
reg = 0.01

# Train a decision tree on the training set
#clf1 = LogisticRegression(C=1/reg).fit(X_train, Y_train)
clf1 = tree.DecisionTreeClassifier()
clf1 = clf1.fit(X_train, Y_train)
print (clf1)

# Evaluate the test set
accuracy = clf1.score(X_test, Y_test)

print ("Accuracy is {}".format(accuracy))

# Serialize the model and write to disk
f = open('model.pkl', 'wb')
pickle.dump(clf1, f)
f.close()
print ("Exported the model to model.pkl")

### Step 2: Test Trained Model

In [ ]:
print("Import the model from model.pkl")
f2 = open('model.pkl', 'rb')

clf2 = pickle.load(f2)

# Normal (not an anomaly)
#X_new = [[24.90294136, 1.44463889, 20.89537849, 24]]
#X_new = [[33.40859853, 2.413637808, 20.89162813, 26]]
#X_new = [[34.42109181, 2.528985143, 21.23903786, 25]]

# Anomaly
X_new = [[33.66995566, 2.44341267, 21.39450979, 26]]
#X_new = [[105.5457931, 10.63179922, 20.62029994, 26]]

print ('New sample: {}'.format(X_new))

pred = clf2.predict(X_new)
print('Predicted class is {}'.format(pred))

### Step 3 Register Model

In [ ]:
from azureml.core.model import Model
override = False
register = True
model_name = "azureml-anormaly-model"
model_path = "model.pkl"

if register:
    model = register_model(ws, model_path, model_name)

### Step 4: Create Inference Script

In [ ]:
%%writefile iot_score.py
# This script generates the scoring file
# with the init and run functions needed to 
# operationalize the anomaly detection sample

import pickle
import json
import pandas
from sklearn.externals import joblib
from sklearn.linear_model import Ridge
from azureml.core.model import Model

def init():
    global model
    model_name = "azureml-anormaly-model"
    # this is a different behavior than before when the code is run locally, even though the code is the same.
    model_path = Model.get_model_path(model_name)
    # deserialize the model file back into a sklearn model
    model = joblib.load(model_path)

# note you can pass in multiple rows for scoring
def run(input_str):
    try:
        input_json = json.loads(input_str)
        input_df = pandas.DataFrame([[input_json['machine']['temperature'],input_json['machine']['pressure'],input_json['ambient']['temperature'],input_json['ambient']['humidity']]])
        pred = model.predict(input_df)
        print("Prediction is ", pred[0])
    except Exception as e:
        result = str(e)
        return json.dumps(result)
        
    if pred[0] == 1:
        input_json['anomaly']=True
    else:
        input_json['anomaly']=False
        
    return [json.dumps(input_json)]
